<a href="https://colab.research.google.com/github/minjaekim070/Bootcamp_Project/blob/main/CNN_project_2%EC%9D%BC%EC%B0%A8_augmentatin%EC%9D%B4%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

전체 데이터셋의 다운로드가 오래 걸려서 bottle 폴더에 대한 내용을 우선적으로 정상/비정상으로 분류할 수 있도록 진행

#Library import


In [2]:
# 패키지 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets, utils
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing import image


#이미지 업로드 및 배열 변환


In [3]:
# 이미지 경로와 카테고리 설정
data_dir = '/content/drive/MyDrive/bottle/train/good'

# 이미지 로딩 및 배열 변환
train_image_data = []
for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    train_image_data.append(img_array)


In [4]:
# 배열로 변환
train_image_data = np.array(train_image_data)
train_image_data = train_image_data / 255

In [5]:
train_image_data.shape

(209, 32, 32, 3)

In [6]:
large_data_dir = '/content/drive/MyDrive/bottle/test/broken_large'
small_data_dir = '/content/drive/MyDrive/bottle/test/broken_small'
contamination_data_dir='/content/drive/MyDrive/bottle/test/contamination'
test_data_dir = '/content/drive/MyDrive/bottle/test/good'

# 이미지 로딩 및 전처리를 위한 ImageDataGenerator 생성
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화합니다

# test 이미지 로딩 및 배열 변환
test_image_data = []
for img_name in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    test_image_data.append(img_array)

large_image_data = []
for img_name in os.listdir(large_data_dir):
    img_path = os.path.join(large_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    large_image_data.append(img_array)

small_image_data = []
for img_name in os.listdir(small_data_dir):
    img_path = os.path.join(small_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    small_image_data.append(img_array)

contamination_image_data = []
for img_name in os.listdir(contamination_data_dir):
    img_path = os.path.join(contamination_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    contamination_image_data.append(img_array)

In [7]:
# 배열로 변환
test_image_data = np.array(test_image_data)
small_image_data = np.array(small_image_data)
large_image_data = np.array(large_image_data)
contamination_image_data = np.array(contamination_image_data)

test_image_data = test_image_data / 255
small_image_data = small_image_data / 255
large_image_data = large_image_data / 255
contamination_image_data = contamination_image_data / 255

In [8]:
test_image_data[0][16][16]

array([0.1882353 , 0.16470589, 0.2       ], dtype=float32)

#데이터 전처리

크게 파손, 적게 파손, 오염된 데이터 셋 모두 augmentation을 통해 데이터의 양을 늘린 뒤에, train과 test로 분리해서 각각의 test셋에 대한 예측 후에 결과 확인

In [ ]:
#배열 변환된 데이터 셋
"""
train_image_data : 정상 훈련 데이터 (200)
test_image_data : 정상 테스트 데이터 (20)
small_image_data : 적게 파손 (20)
large_image_data : 크게 파손 (20)
contamination_image_data : 오염 (20)
"""

In [107]:
subset_train_data = train_image_data[:50]

In [108]:
# 이미지 데이터 통합
all_images = np.concatenate((subset_train_data, large_image_data, contamination_image_data), axis=0)

# 이미지 리사이즈
resized_images = []
for image in all_images:
    resized_image = tf.image.resize(image, (32, 32))
    resized_images.append(resized_image)

# 이미지 데이터 변환
resized_images = np.array(resized_images)
resized_images = resized_images.astype(np.float32)

# 레이블 생성
train_labels = np.array([[1, 0, 0]] * len(subset_train_data))  # 정상: [1, 0, 0]
broken_labels = np.array([[0, 1, 0]] * len(large_image_data))  # 파손: [0, 1, 0]
contamination_labels = np.array([[0, 0, 1]] * len(contamination_image_data))  # 오염: [0, 0, 1]

all_labels = np.concatenate((train_labels, broken_labels, contamination_labels), axis=0)


In [109]:
print(resized_images.shape)
print(all_labels.shape)

(91, 32, 32, 3)
(91, 3)


#모델 구성 및 결과 예측

In [113]:
import tensorflow as tf
from tensorflow.keras import layers, models

train_labels = [[1, 0, 0],   # 첫 번째 이미지의 레이블: [정상, 파손, 오염] = [1, 0, 0]
                [0, 1, 0],   # 두 번째 이미지의 레이블: [정상, 파손, 오염] = [0, 1, 0]
                [0, 0, 1]]  # 세 번째 이미지의 레이블: [정상, 파손, 오염] = [0, 0, 1]
# CNN 모델 구성
model = models.Sequential()

#Layer 1
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

#Layer 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#Layer 3
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

#1D
model.add(layers.Flatten())

#Dense
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약 출력
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 2, 2, 128)       

In [114]:
# 모델 학습
model.fit(resized_images, all_labels, epochs=20, batch_size=32)

Epoch 1/20
3/3 [==============================] - 2s 8ms/step - loss: 1.0370 - accuracy: 0.5055
Epoch 2/20
3/3 [==============================] - 0s 7ms/step - loss: 1.0058 - accuracy: 0.5495
Epoch 3/20
3/3 [==============================] - 0s 6ms/step - loss: 1.0066 - accuracy: 0.5495
Epoch 4/20
3/3 [==============================] - 0s 6ms/step - loss: 0.9978 - accuracy: 0.5495
Epoch 5/20
3/3 [==============================] - 0s 7ms/step - loss: 0.9982 - accuracy: 0.5495
Epoch 6/20
3/3 [==============================] - 0s 11ms/step - loss: 0.9821 - accuracy: 0.5495
Epoch 7/20
3/3 [==============================] - 0s 7ms/step - loss: 0.9808 - accuracy: 0.5495
Epoch 8/20
3/3 [==============================] - 0s 7ms/step - loss: 0.9770 - accuracy: 0.5714
Epoch 9/20
3/3 [==============================] - 0s 6ms/step - loss: 0.9551 - accuracy: 0.5495
Epoch 10/20
3/3 [==============================] - 0s 7ms/step - loss: 0.9472 - accuracy: 0.5495
Epoch 11/20
3/3 [=====================

In [115]:
# 예측 수행
predictions = model.predict(small_image_data)

1/1 [==============================] - 0s 75ms/step


In [116]:
# 최종 예측 레이블 생성
predicted_labels = []
for prediction in predictions:
    if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
        predicted_labels.append([1, 0, 0])
    elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
        predicted_labels.append([0, 1, 0])
    else:
        predicted_labels.append([0, 0, 1])
predicted_labels = np.array(predicted_labels)
count = 0
for label in predicted_labels:
    if np.array_equal(label, [0, 1, 0]):
        count += 1

prediction_accuracy = count / len(small_image_data) * 100
print('예측률: {}%'.format(prediction_accuracy))

예측률: 50.0%


In [117]:
predictions = model.predict(test_image_data)
predicted_labels = []
for prediction in predictions:
    if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
        predicted_labels.append([1, 0, 0])
    elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
        predicted_labels.append([0, 1, 0])
    else:
        predicted_labels.append([0, 0, 1])
predicted_labels = np.array(predicted_labels)

count = 0
for label in predicted_labels:
    if np.array_equal(label, [1, 0, 0]):
        count += 1

prediction_accuracy = count / len(test_image_data) * 100
print('예측률: {}%'.format(prediction_accuracy))


1/1 [==============================] - 0s 19ms/step
예측률: 100.0%
